In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\4\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\4\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[39.0, 40.0, 84.0, 37.0, 37.0, 86.0, 100.0, 32.0, 84.0, 100.0, 95.0, 66.0, 66.0, 47.0, 47.0, 45.0, 32.0, 7.0, 86.0, 80.0, 84.0, 100.0, 13.0, 52.0, 32.0, 30.0, 66.0, 37.0, 37.0, 4.0, 86.0, 7.0, 13.0, 13.0, 84.0, 7.0, 37.0, 84.0, 4.0, 32.0, 40.0, 93.0, 45.0, 93.0, 100.0, 40.0, 16.0, 47.0, 7.0, 52.0, 84.0, 80.0, 93.0, 80.0, 96.0, 20.0, 35.0, 10.0, 37.0, 7.0, 45.0, 16.0, 80.0, 84.0, 32.0, 16.0, 80.0, 13.0, 13.0, 39.0, 47.0, 4.0, 16.0, 35.0, 47.0, 4.0, 47.0, 45.0, 55.0, 86.0, 55.0, 37.0, 66.0, 37.0, 16.0, 37.0, 95.0, 47.0, 96.0, 66.0, 93.0, 7.0, 100.0, 66.0, 39.0, 100.0, 95.0, 4.0, 7.0, 16.0, 45.0, 30.0, 84.0, 45.0, 47.0, 96.0, 32.0, 100.0, 32.0, 23.0, 10.0, 66.0, 84.0, 52.0, 80.0, 100.0, 7.0, 66.0, 16.0, 13.0, 40.0, 13.0, 66.0, 66.0, 32.0, 23.0, 86.0, 4.0, 35.0, 93.0, 52.0, 66.0, 66.0, 52.0, 45.0, 16.0, 86.0, 96.0, 80.0, 30.0, 80.0, 47.0, 47.0, 84.0, 7.0, 13.0, 7.0, 52.0, 13.0, 45.0, 37.0, 45.0, 100.0, 7.0, 13.0, 23.0, 32.0, 20.0, 80.0, 80.0, 95.0, 84.0, 55.0, 13.0, 13.0, 66.0, 96.0, 4.0, 

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
RegNetY002 =tf.keras.applications.regnet.RegNetY002(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY002.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY002(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety002 (Functional)     (None, 7, 7, 368)         2814844   
                                                                 
 global_average_pooling2d (G  (None, 368)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 369       
                                                                 
Total params: 2,815,213
Trainable params: 2,794,365
Non-trainable params: 20,848
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY002_loss_4.txt',history.history['loss'])
#要改
np.savetxt('RegNetY002_valloss_4.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Fri Apr 14 17:02:24 2023
Epoch 1/300
592/592 [==============================] - ETA: 0s - loss: 620.9741

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt\assets


592/592 [==============================] - 75s 102ms/step - loss: 620.9741 - val_loss: 14.3970
Epoch 2/300
592/592 [==============================] - ETA: 0s - loss: 15.8870

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt\assets


592/592 [==============================] - 104s 176ms/step - loss: 0.3690 - val_loss: 0.1644
Epoch 254/300
592/592 [==============================] - 84s 142ms/step - loss: 0.4438 - val_loss: 0.2381
Epoch 255/300
592/592 [==============================] - 86s 144ms/step - loss: 0.4245 - val_loss: 0.3005
Epoch 256/300
592/592 [==============================] - 86s 146ms/step - loss: 0.4023 - val_loss: 0.2438
Epoch 257/300
592/592 [==============================] - 86s 144ms/step - loss: 0.3628 - val_loss: 0.2677
Epoch 258/300
592/592 [==============================] - 85s 143ms/step - loss: 0.4283 - val_loss: 0.2383
Epoch 259/300
592/592 [==============================] - 85s 144ms/step - loss: 0.3619 - val_loss: 0.2453
Epoch 260/300
592/592 [==============================] - 85s 142ms/step - loss: 0.3752 - val_loss: 0.2138
Epoch 261/300
592/592 [==============================] - 87s 146ms/step - loss: 0.3798 - val_loss: 0.2228
Epoch 262/300
592/592 [==============================] - 86

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\4\RegNetY002.ckpt\assets


592/592 [==============================] - 107s 181ms/step - loss: 0.3895 - val_loss: 0.1633
Epoch 298/300
592/592 [==============================] - 89s 149ms/step - loss: 0.3870 - val_loss: 0.2343
Epoch 299/300
592/592 [==============================] - 89s 151ms/step - loss: 0.3600 - val_loss: 0.2598
Epoch 300/300
592/592 [==============================] - 89s 150ms/step - loss: 0.3599 - val_loss: 0.2678
end time: Fri Apr 14 23:10:26 2023
